No importa su habilidad como programador, eventualmente cometerá un error de codificación. Tales errores vienen en tres sabores básicos:

**Errores de sintaxis**: errores en los que el código no es Python válido (generalmente es fácil de corregir)

**Errores en tiempo de ejecución**: errores en los que el código sintácticamente válido no se ejecuta, quizás debido a una entrada de usuario no válida (a veces es fácil de corregir)

**Errores semánticos**: errores en la lógica: el código se ejecuta sin problemas, pero el resultado no es el esperado (a menudo es muy difícil de rastrear y corregir)

Aquí nos enfocaremos en cómo lidiar limpiamente con los errores de tiempo de ejecución . Como veremos, Python maneja los errores en tiempo de ejecución a través de su marco de manejo de excepciones .M

#### Errores en tiempo de ejecución 
Si ha realizado alguna codificación en Python, es probable que se haya encontrado con errores de tiempo de ejecución. Pueden suceder de muchas formas.

Por ejemplo, si intenta hacer referencia a una variable indefinida:

In [3]:
print(q)

NameError: name 'q' is not defined

O si prueba una operación que no está definida:

In [5]:
1 + 'ab'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

O puede estar intentando calcular un resultado matemáticamente mal definido:



In [7]:
2/0

ZeroDivisionError: division by zero

O tal vez está intentando acceder a un elemento de secuencia que no existe:

In [9]:
L  =  [ 1 ,  2 ,  3 ] 
L [ 1000 ]

IndexError: list index out of range

Tenga en cuenta que en cada caso, Python tiene la amabilidad de no simplemente indicar que ocurrió un error, sino de escupir una excepción significativa que incluye información sobre qué salió mal exactamente, junto con la línea exacta de código donde ocurrió el error. Tener acceso a errores significativos como este es inmensamente útil cuando se intenta rastrear la raíz de los problemas en su código.

In [13]:
try:
    print("this gets executed first")
except:
    print("this gets executed only if there is an error")

this gets executed first


Tenga en cuenta que el segundo bloque aquí no se ejecutó: esto se debe a que el primer bloque no devolvió un error. Pongamos una declaración problemática en el trybloque y veamos qué sucede:

In [14]:
try:
    print("let's try something:")
    x = 1 / 0 # ZeroDivisionError
except:
    print("something bad happened!")

let's try something:
something bad happened!


Aquí vemos que cuando se generó el error en la try declaración (en este caso, a ZeroDivisionError), se detectó el error y except se ejecutó la declaración.

Una forma en que esto se usa a menudo es verificar la entrada del usuario dentro de una función u otro fragmento de código. Por ejemplo, es posible que deseemos tener una función que capture la división cero y devuelva algún otro valor, tal vez un número suficientemente grande como 10^100:

In [15]:
def safe_divide(a, b):
    try:
        return a / b
    except:
        return 1E100

In [16]:
safe_divide(1, 2)

0.5

In [18]:
safe_divide(2,0)

1e+100

Sin embargo, hay un problema sutil con este código: ¿qué sucede cuando aparece otro tipo de excepción? Por ejemplo, probablemente esto no sea lo que pretendíamos:

In [19]:
safe_divide (1, '2')

1e+100

Dividir un entero y una cadena genera un TypeError, que nuestro código demasiado entusiasta captó y asumió que era un ZeroDivisionError! Por esta razón, casi siempre es una mejor idea detectar las excepciones de forma explícita :

In [22]:
def safe_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return 1E100

In [23]:
safe_divide(2,0)

1e+100

In [24]:
safe_divide(3,'1')

TypeError: unsupported operand type(s) for /: 'int' and 'str'

Ahora solo detectamos errores de división cero y permitimos que todos los demás errores pasen sin modificar.

##### Generación de excepciones: raise

Hemos visto lo valioso que es tener excepciones informativas al usar partes del lenguaje Python. Es igualmente valioso hacer uso de excepciones informativas dentro del código que escribe, para que los usuarios de su código (¡sobre todo usted mismo!) Puedan descubrir qué causó sus errores.

La forma en que plantea sus propias excepciones es con la declaración raise. Por ejemplo:

In [27]:
raise  RuntimeError ( "mi mensaje de error" )

RuntimeError: mi mensaje de error

Como ejemplo de dónde esto podría ser útil, regresemos a nuestra función fibonacci que definimos anteriormente:

In [30]:
def fibonacci(N):
    L = []
    a, b = 0, 1
    while len(L) < N:
        a, b = b, a + b
        L.append(a)
    return L

Un problema potencial aquí es que el valor de entrada podría ser negativo. Actualmente, esto no causará ningún error en nuestra función, pero es posible que deseemos que el usuario sepa que N no admite un negativo . Los errores derivados de valores de parámetros no válidos, por convención, llevan a que ValueError se genere:

In [31]:
fibonacci(-12)

[]

In [34]:
def fibonacci(N):
    if N < 0:
        raise ValueError("N no debe ser negativo")
    L = []
    a, b = 0, 1
    while len(L) < N:
        a, b = b, a + b
        L.append(a)
    return L

In [35]:
fibonacci(-90)

ValueError: N no debe ser negativo

Ahora el usuario sabe exactamente por qué la entrada no es válida, ¡e incluso podría usar un bloque try... except para manejarla!

In [37]:
N = -10
try:
    print("trying this...")
    print(fibonacci(N))
except ValueError:
    print("Bad value: need to do something else")

trying this...
Bad value: need to do something else


#### Profundizando en las excepciones 
Brevemente, quiero mencionar aquí algunos otros conceptos con los que podría encontrarse. No entraré en detalles sobre estos conceptos y cómo y por qué usarlos, sino que simplemente le mostraré la sintaxis para que pueda explorar más por su cuenta.

###### Accediendo al mensaje de error 
A veces, en una declaración try... except, le gustaría poder trabajar con el mensaje de error en sí. Esto se puede hacer con la palabra clave as:

In [42]:
try:
    x = 1 / 0
except ZeroDivisionError as err:
    print("Error class is:  ", type(err))
    print("Error message is:", err)

Error class is:   <class 'ZeroDivisionError'>
Error message is: division by zero


Con este patrón, puede personalizar aún más el manejo de excepciones de su función.

##### Definición de excepciones personalizadas 
Además de las excepciones integradas, es posible definir excepciones personalizadas a través de la herencia de clases . Por ejemplo, si desea un tipo especial de ValueError, puede hacer esto:

In [43]:
class MySpecialError(ValueError):
    pass

raise MySpecialError("here's the message")

MySpecialError: here's the message

Esto le permitiría usar un bloque try... exceptque solo detecta este tipo de error:

In [45]:
try:
    print("do something")
    raise MySpecialError("[informative error message here]")
except MySpecialError:
    print("do something else")

do something
do something else


Puede que le resulte útil a medida que desarrolle un código más personalizado.

try... except... else... finally
Además de try y except, puede utilizar las palabras clave else y finally para ajustar aún más el manejo de excepciones de su código. La estructura básica es esta:

In [48]:
try:
    print("try something here")
except:
    print("this happens only if it fails")
else:
    print("this happens only if it succeeds")
finally:
    print("this happens no matter what")

try something here
this happens only if it succeeds
this happens no matter what


La utilidad de elsea quí es clara, pero ¿de qué sirve finally? Bueno, la finally cláusula realmente se ejecuta sin importar qué : generalmente veo que se usa para hacer algún tipo de limpieza después de que se completa una operación.